## Event Loop

An **event loop** is a programming construct that enables a program to handle multiple tasks concurrently within a single thread. It works by continuously checking for and dispatching events or messages. This model is the cornerstone of non-blocking I/O and asynchronous programming, allowing an application to remain responsive instead of getting stuck waiting for slow operations to complete.

Think of it like a chef in a kitchen working alone. 🧑‍🍳

  * A **synchronous (blocking) chef** would take one order, prepare the entire dish from start to finish, and serve it before even looking at the next order. If a steak takes 10 minutes to grill, all other customers must wait.
  * An **asynchronous chef (using an event loop)** would take an order, put the steak on the grill (a slow task), and while it's grilling, they would start chopping vegetables for the next order. They continuously "loop" through their tasks, checking if the steak is ready, if the water is boiling, etc. This chef (the single thread) is never idly waiting and can make progress on many dishes at once.

-----

### How It Works: The Core Components

An event loop primarily consists of two parts: the **Event Queue** and the **Loop** itself.

1.  **The Event Queue**: This is a simple first-in, first-out (FIFO) queue where events are registered. An event can be anything from a user clicking a button, a timer expiring, or data arriving from a network request.

2.  **The Loop**: This is an infinite loop that performs a simple, repetitive two-step process:

      * **Check**: It checks the event queue for any pending events.
      * **Execute**: If an event exists, it takes the first one from the queue and executes its associated function (often called a "callback" or "handler"). Once the handler finishes, the loop repeats.

If the queue is empty, the loop simply waits until a new event is added. This prevents the single thread from being blocked by any one task. When a task involves waiting (e.g., for a file download), it is handed off, and the event loop is free to process other events. Once the download is complete, a "download finished" event is placed in the queue for the loop to process later.

-----

### Python Example with `asyncio`

Python's `asyncio` library is a prime example of an event-driven framework. The keywords `async` and `await` are used to work with the event loop.

  * `async def`: Defines a function as a **coroutine**. This is a special function that can be paused and resumed.
  * `await`: Pauses the execution of the coroutine and tells the event loop, "I'm waiting for this result, but you can go do other work in the meantime."

In the code below, we simulate fetching data from two different sources with different delays. Instead of taking 2 + 3 = 5 seconds, the program runs them concurrently, and the total time is only slightly more than the longest task (3 seconds).

```python
import asyncio
import time

async def fetch_data(source: str, delay: int):
    """A coroutine that simulates a slow network request."""
    print(f"▶️  Starting to fetch from {source}...")
    
    # await asyncio.sleep() is a non-blocking pause.
    # It cedes control back to the event loop, allowing other tasks to run.
    await asyncio.sleep(delay)
    
    print(f"✅ Finished fetching from {source}.")
    return {"source": source, "data": f"Sample data after {delay}s"}

async def main():
    """The main coroutine to run our tasks concurrently."""
    start_time = time.time()
    print(f"Program started at {time.strftime('%X')}")

    # asyncio.gather() schedules both coroutines to run on the event loop.
    # The loop will intelligently switch between them whenever one 'awaits'.
    task1 = fetch_data("API Server", 2)
    task2 = fetch_data("Database", 3)
    
    results = await asyncio.gather(task1, task2)

    end_time = time.time()
    print(f"Program finished at {time.strftime('%X')}")
    print(f"Total execution time: {end_time - start_time:.2f} seconds")
    print("--- Results ---")
    for res in results:
        print(res)

# asyncio.run() starts the event loop and runs the main() coroutine until it's done.
# It handles starting, managing, and tearing down the loop.
if __name__ == "__main__":
    asyncio.run(main())
```

#### **Expected Output:**

```
Program started at 08:31:44
▶️  Starting to fetch from API Server...
▶️  Starting to fetch from Database...
✅ Finished fetching from API Server.
✅ Finished fetching from Database.
Program finished at 08:31:47
Total execution time: 3.01 seconds
--- Results ---
{'source': 'API Server', 'data': 'Sample data after 2s'}
{'source': 'Database', 'data': 'Sample data after 3s'}
```

Notice how both "Starting to fetch" messages appear immediately, and the total time is \~3 seconds, not 5. This is the power of the event loop efficiently managing waiting time.

-----

### Common Use Cases

The event loop architecture is fundamental to many modern technologies:

  * **JavaScript (Node.js & Browsers)**: The entire asynchronous nature of JavaScript, from handling user clicks (`onClick`) to making `fetch` requests, is built upon an event loop.
  * **High-Performance Web Servers**: Python frameworks like **FastAPI** and **Tornado**, as well as Node.js, use event loops to handle thousands of simultaneous network connections efficiently.
  * **Graphical User Interfaces (GUIs)**: GUI toolkits use an event loop to respond to user actions like mouse clicks and key presses without freezing the entire application window.

<img src="./pics/event_loop.png"  width="800" height="500">